In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate

from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, ElasticNet, ElasticNetCV # Линейна регрессия, Ридж и Лассо
from sklearn.linear_model import LassoCV, RidgeCV, MultiTaskLassoCV # Ridge и Lasso Регрессия
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics import make_scorer, r2_score

from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

import gc

sns.set()
%matplotlib inline

import sys
# np.set_printoptions(suppress=True)
# np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=3)

DISPLAY_MAX_ROWS = 20 #20
pd.set_option('display.max_rows', DISPLAY_MAX_ROWS)
pd.set_option('display.max_column',None)
plt.style.use('seaborn-whitegrid')


# plt.rcParams["figure.figsize"] = (20, 15)

import warnings
warnings.filterwarnings('ignore')

<ipython-input-1-41f3d8d78b0d>:36: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [2]:
df = pd.read_csv("data.csv")

In [3]:
df

,KS4_GPA5_Qtg.PV,KS4_GPA5_TC_T1.PV,KS4_GPA5_Nst.PV,KS4_GPA5_Nvd.PV,KS4_GPA5_Nnd.PV,KS4_GPA5_T4.PV,KS4_GPA5_Pk.PV
0,50.074285,50.656771,500.475537,1207.542773,912.161426,119.562970,51.248955
1,50.074926,50.656771,500.379883,1209.190723,914.203027,119.683643,51.255357
2,50.074836,50.666920,499.359570,1210.159082,914.924512,119.665576,51.263570
3,50.074957,50.666920,500.535840,1210.641602,915.881055,119.674750,51.267725
4,50.075145,50.666920,500.198145,1211.398438,916.435645,119.692188,51.264826
...,...,...,...,...,...,...,...
29707,50.070496,52.706500,459.827393,1238.176660,914.571973,128.579962,51.102349
29708,50.070217,52.716716,459.400806,1237.532422,913.476074,128.521655,51.102543
29709,50.070292,52.696284,459.942578,1237.783984,913.913086,128.503632,51.102639
29710,50.070389,52.675853,460.185791,1237.441895,913.895508,128.481744,51.103799


In [4]:
inputs = ['KS4_GPA5_Qtg.PV', 'KS4_GPA5_TC_T1.PV', 'KS4_GPA5_Nst.PV'] # предикторы
outputs = ['KS4_GPA5_Nvd.PV', 'KS4_GPA5_Nnd.PV', 'KS4_GPA5_T4.PV', 'KS4_GPA5_Pk.PV'] # отклики, целевые признаки

# Задача - построить предсказательую модель outputs по inputs
### Результатом решения задачи является код эксперимента в Jupyter Notebook, содержащий качественное обоснование выбора той или иной архитектуры (метода). В качестве base line рекомендуется выбрать модель линейной регрессии.

# ДЗ:
1. Сделать разведочный анализ данных, найти пропуски и выбросы. Примеры есть в ноутбуке EDA_AUTO из 4 задания.
2. Заполнить пропуски. Заполнить медианой или обучить линейную регрессию и заполнить ею пропуски. Если заполните линейной регрессией пропуски, то с помощью таблицы корреляций найдите второй признак, который имеет сильную взаимосвязь с первым признаком. Создаете отдельный датафрейм из этих признаков, исключаете пропуски (не удаляете), обучаете линейну регрессию. Затем прогнозируете на исключенных данных с пропусками и заполняете эти пропуски предсказанным значением.
3. Обучить множественную линейную регрессию. Она будет являться base line, т.е. базовая модель. С ней вы будете сравнивать другие модели.
4. Используя кроссвалидацию, подбор гиперпараметров обучить модели Lasso regression, Ridge Regression, ElasticNet (модель где Ridge + Lasso вместе используется), DecisionTreeRegressor, SVR, RandomForestRegressor

5. `alphas=[0.0001, 0.001,0.01, 0.1, 1, 10]` - для MultiTaskLassoCV, RidgeCV, и для ElasticNetCV.

`'l1_ratio':[0.01, 0.1, 0.5, 1, 5, 10, 15, 20]` - для ElasticNetCV

6. **для SVR и RandomForest:**
```python
param_grid = [{'regressor':[SVR()], 'preprocessing':[StandardScaler(), None],
              'regressor__gamma':[0.001, 0.01, 0.1, 1, 10, 100],
               'regressor__C': [0.001, 0.01, 0.1, 1, 10, 100]},
              {'regressor': [DecisionTreeRegressor()],
               'preprocessing': [None],
               'regressor__max_features': [1, 2, 3]},
              {'regressor': [RandomForestRegressor(n_estimators=100)],
               'preprocessing': [None],
               'regressor__max_features': [1, 2, 3]}]
```

7. **Если SVR обучается, то используем только этот набор параметров:**
```python
param_grid={
    'estimator__gamma': [0.001, 0.01],
    'estimator__C': [0.001, 0.01],
}
```

In [5]:
df.describe()

,KS4_GPA5_Qtg.PV,KS4_GPA5_TC_T1.PV,KS4_GPA5_Nst.PV,KS4_GPA5_Nvd.PV,KS4_GPA5_Nnd.PV,KS4_GPA5_T4.PV,KS4_GPA5_Pk.PV
count,29712.000000,29712.000000,29712.000000,29712.000000,27711.000000,29712.000000,29712.000000
mean,50.075761,51.582940,496.375057,1228.936047,925.288586,123.147985,51.579620
std,0.004490,0.718957,16.656003,9.293723,12.649640,2.293297,58.430508
min,50.067757,50.118885,458.647290,1207.311621,910.000195,118.375299,51.074956
25%,50.072440,50.940938,484.903076,1222.372168,915.022510,121.428400,51.183080
50%,50.074476,51.725804,490.411133,1228.801709,920.266797,123.233508,51.244438
75%,50.079485,52.134428,509.792322,1236.431030,936.632568,125.019626,51.271976
max,50.089918,53.053829,526.688867,1264.442773,977.826953,128.815308,10123.000000


In [6]:
df['KS4_GPA5_Pk.PV'].sort_values(ascending=False)

20100    10123.000000
7611        51.402325
7613        51.401648
7623        51.401648
7633        51.401358
             ...     
29664       51.096454
29671       51.096261
29672       51.095778
29674       51.095585
28419       51.074956
Name: KS4_GPA5_Pk.PV, Length: 29712, dtype: float64

In [7]:
df.at[20100, 'KS4_GPA5_Pk.PV'] = df['KS4_GPA5_Pk.PV'].median()

In [8]:
print(df.isnull().any())
print(df.isna().any())

KS4_GPA5_Qtg.PV      False
KS4_GPA5_TC_T1.PV    False
KS4_GPA5_Nst.PV      False
KS4_GPA5_Nvd.PV      False
KS4_GPA5_Nnd.PV       True
KS4_GPA5_T4.PV       False
KS4_GPA5_Pk.PV       False
dtype: bool
KS4_GPA5_Qtg.PV      False
KS4_GPA5_TC_T1.PV    False
KS4_GPA5_Nst.PV      False
KS4_GPA5_Nvd.PV      False
KS4_GPA5_Nnd.PV       True
KS4_GPA5_T4.PV       False
KS4_GPA5_Pk.PV       False
dtype: bool


In [9]:
print(df.isnull().sum())

KS4_GPA5_Qtg.PV         0
KS4_GPA5_TC_T1.PV       0
KS4_GPA5_Nst.PV         0
KS4_GPA5_Nvd.PV         0
KS4_GPA5_Nnd.PV      2001
KS4_GPA5_T4.PV          0
KS4_GPA5_Pk.PV          0
dtype: int64


In [10]:
df.fillna(df.mean(), inplace=True)

In [11]:
print(df.isnull().sum())

KS4_GPA5_Qtg.PV      0
KS4_GPA5_TC_T1.PV    0
KS4_GPA5_Nst.PV      0
KS4_GPA5_Nvd.PV      0
KS4_GPA5_Nnd.PV      0
KS4_GPA5_T4.PV       0
KS4_GPA5_Pk.PV       0
dtype: int64


In [12]:
X = df[inputs]
y = df[outputs]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Linear

In [31]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Оценка точности модели: {model.score(X_train, y_train), model.score(X_test, y_test)}')

Оценка точности модели: (0.8780119233207486, 0.87909829849615)


# Lasso

In [29]:
param_grid={"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]}
model = GridSearchCV(Lasso(), param_grid,cv=5, n_jobs=-1)
model.fit(X_train, y_train)

print(f'Оценка точности модели: {model.score(X_train, y_train), model.score(X_test, y_test)}')
print("Наилучшее значение параметра:", model.best_estimator_)

Оценка точности модели: (0.873763749979388, 0.8748159717286448)
Наилучшее значение параметра: Lasso(alpha=0.0001)


# Ridge

In [27]:
model = GridSearchCV(Ridge(), param_grid,cv=5,n_jobs=-1)
model.fit(X_train, y_train)

print(f'Оценка точности модели: {model.score(X_train, y_train), model.score(X_test, y_test)}')
print("Наилучшее значение параметра:", model.best_estimator_)

Оценка точности модели: (0.878011722846603, 0.8790976918687725)
Наилучшее значение параметра: Ridge(alpha=0.0001)


# ElasticNet

In [21]:
param_grid_elasticnet = param_grid.copy()
param_grid_elasticnet['l1_ratio'] = [0.01, 0.1, 0.5, 1, 5, 10, 15, 20]
model = GridSearchCV(ElasticNet(), param_grid_elasticnet, cv=5,n_jobs=-1)
model.fit(X_train, y_train)

print(f'Оценка точности модели: {model.best_estimator_.score(X_train, y_train), model.best_estimator_.score(X_test, y_test)}')
print("Наилучшее значение параметров:", model.best_estimator_)

Оценка точности модели: (0.873763749979388, 0.8748159717286448)
Наилучшее значение параметров: ElasticNet(alpha=0.0001, l1_ratio=1)


# DecisionTree

In [23]:
param_grid_dt = {'max_features': [1, 2, 3]}
model = GridSearchCV(DecisionTreeRegressor(), param_grid_dt,cv=5,n_jobs=-1)
model.fit(X_train, y_train)
print(f'Оценка точности модели: {model.score(X_train, y_train), model.score(X_test, y_test)}')
print("Наилучшее значение параметров: ", model.best_params_)

Оценка точности модели: (0.9999999310367996, 0.9845348606909508)
Наилучшее значение параметров:  {'max_features': 3}


# SVR

In [24]:
from sklearn.multioutput import MultiOutputRegressor
param_grid_svr = {
    'estimator__gamma': [0.001, 0.01],
    'estimator__C': [0.001, 0.01],
}
svr = SVR()
multioutput_svr = MultiOutputRegressor(svr)
model = GridSearchCV(multioutput_svr, param_grid_svr,cv=3,n_jobs=-1)
model.fit(X_train, y_train)

print(f'Оценка точности модели: {model.score(X_train, y_train), model.score(X_test, y_test)}')
print("Наилучшее значение параметров: ", model.best_params_)

Оценка точности модели: (0.5113365346587987, 0.5105051800881988)
Наилучшее значение параметров:  {'estimator__C': 0.01, 'estimator__gamma': 0.01}


# RandomForest

In [25]:
param_grid_rf = {'max_features': [1, 2, 3]}
model = GridSearchCV(DecisionTreeRegressor(), param_grid_rf,cv=5,n_jobs=-1)
model.fit(X_train, y_train)

print(f'Оценка точности модели: {model.best_estimator_.score(X_train, y_train), model.best_estimator_.score(X_test, y_test)}')
print("Наилучшее значение параметров: ", model.best_estimator_)

Оценка точности модели: (0.9999999310367996, 0.9846245153412055)
Наилучшее значение параметров:  DecisionTreeRegressor(max_features=3)
